# Conversational agent

In [57]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")

In [58]:
from langchain.tools import tool

In [59]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [60]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [61]:
tools = [get_current_temperature, search_wikipedia]

In [62]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [63]:
functions = [format_tool_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [64]:
result = chain.invoke({"input": "what is the weather is sf?"})

In [65]:
result.tool

'get_current_temperature'

In [66]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [67]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [68]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [69]:
result1 = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": []
})

In [14]:
result1.tool

'get_current_temperature'

In [70]:
observation = get_current_temperature(result1.tool_input)

C:\Users\luca5\AppData\Local\Temp\ipykernel_40700\4059959254.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


In [71]:
observation

'The current temperature is 20.8°C'

In [72]:
type(result1)

langchain_core.agents.AgentActionMessageLog

In [74]:
result1

AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 37.7749, 'longitude': -122.4194}, log="\nInvoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 112, 'total_tokens': 137, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-7bb7dab6-beef-43f4-bc05-c0eaf6ee4fd9-0')])

In [73]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [19]:
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 112, 'total_tokens': 137, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-7aa18ed7-6919-4e6f-b742-c9a66f09e8b6-0')]

In [20]:
format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 112, 'total_tokens': 137, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-7aa18ed7-6919-4e6f-b742-c9a66f09e8b6-0'),
 FunctionMessage(content='The current temperature is 19.7°C', additional_kwargs={}, response_metadata={}, name='get_current_temperature')]

In [32]:
result2 = chain.invoke({
    "input": "what is the weather in sf?", 
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

In [33]:
result2

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 19.7°C.'}, log='The current temperature in San Francisco is 19.7°C.')

In [34]:
from langchain.schema.agent import AgentFinish
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [76]:
from langchain.schema.runnable import RunnablePassthrough
# ajouter un nouvel élément au dictionnaire d'input avec assign()
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [77]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input, 
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [78]:
run_agent("what is the weather in sf?")

C:\Users\luca5\AppData\Local\Temp\ipykernel_40700\4059959254.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


AgentFinish(return_values={'output': 'The current temperature in San Francisco is 20.8°C.'}, log='The current temperature in San Francisco is 20.8°C.')

In [79]:
run_agent("what is langchain?")

AgentFinish(return_values={'output': 'LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. It is used for document analysis and summarization, chatbots, and code analysis.'}, log='LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. It is used for document analysis and summarization, chatbots, and code analysis.')

In [80]:
run_agent("hi!")

AgentFinish(return_values={'output': 'Well, hello there! How can I assist you today?'}, log='Well, hello there! How can I assist you today?')

In [82]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [83]:
agent_executor.invoke({"input": "what is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Langchain'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Retrieval-augmented generation
Summary: Retrieval augmented generation (RAG) is a type of generative artificial intelligence that has information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information in preference to information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  
Use cases include providing chatbot access to interna

{'input': 'what is langchain?',
 'output': 'LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. It is used for tasks such as document analysis and summarization, chatbots, and code analysis.'}

In [84]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [85]:
agent_executor.invoke({"input": "what is my name"})



> Entering new AgentExecutor chain...
I'm sorry, I don't have access to your personal information. How can I assist you today?

> Finished chain.


{'input': 'what is my name',
 'output': "I'm sorry, I don't have access to your personal information. How can I assist you today?"}

In [87]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [88]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [89]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [90]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [91]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [92]:
agent_executor.invoke({"input": "whats my name"})



> Entering new AgentExecutor chain...
You just told me your name is Bob. So, your name is Bob! How can I assist you further, Bob?

> Finished chain.


{'input': 'whats my name',
 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='whats my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You just told me your name is Bob. So, your name is Bob! How can I assist you further, Bob?', additional_kwargs={}, response_metadata={})],
 'output': 'You just told me your name is Bob. So, your name is Bob! How can I assist you further, Bob?'}

In [93]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`


The current temperature is 20.8°C

C:\Users\luca5\AppData\Local\Temp\ipykernel_40700\4059959254.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


The current temperature in San Francisco is 20.8°C. Is there anything else you would like to know?

> Finished chain.


{'input': 'whats the weather in sf?',
 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='whats my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You just told me your name is Bob. So, your name is Bob! How can I assist you further, Bob?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The current temperature in San Francisco is 20.8°C. Is there anything else you would like to know?', additional_kwargs={}, response_metadata={})],
 'output': 'The current temperature in San Francisco is 20.8°C. Is there anything else you would like to know?'}

### Create a chatbot

In [94]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [95]:
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [ ]:
import panel as pn
pn.extension()  # Extension pour fonctionner dans le notebook
import param  

# Définition de la classe "cbfs" qui gère l'interface de la conversation et les outils
# param.Parameterized est optionnel, on pourrait s'en passer, ainsi que le super() plus bas,
# c'est uniquement si on veut rajouter des paramètres spécifiques, je sais pas trop pourquoi ils l'ont mis là puisqu'ils n'en rajoutent pas
class cbfs(param.Parameterized):  
    # Initialisation de la classe avec une liste d'outils et des paramètres optionnels (keyword arguments)
    def __init__(self, tools, **params):  
        super(cbfs, self).__init__(**params)  # Initialisation de la classe parente (optionnel comme mentionné plus haut)
        self.panels = []  # Liste pour stocker les différents panneaux d'affichage (les bulles du chat en gros)

        self.functions = [format_tool_to_openai_function(f) for f in tools]  

        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)  

        # Mise en place d'une mémoire de conversation pour garder l'historique des messages
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

        # Définition du template de prompt pour gérer la conversation
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),  # Définition du comportement du chatbot
            MessagesPlaceholder(variable_name="chat_history"),  # Placeholder pour l'historique de chat
            ("user", "{input}"),  # Placeholder pour l'entrée de l'utilisateur
            MessagesPlaceholder(variable_name="agent_scratchpad")  # Placeholder pour les étapes intermédiaires (appels de tool)
        ])
        # Chaîne éxécutant l'appel au llm initial qui va décider quel tool utiliser (ou pas)
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])  # Gestion des étapes intermédiaires
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()  # Exécution de la chaîne avec parsing des fonctions OpenAI
        # qa contient l'AgentExecutor avec notre chain, les outils et la mémoire de conversation
        # c'est l'AgentExecutor qui va appeler les tools et renvoyer une réponse à partir de l'historique et de l'output des tools
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)

    # Fonction qui gère la chaîne de conversation, c'est elle qu'on va appeler avec panel après
    def convchain(self, query):
        if not query:  # Vérification si la requête est vide
            return
        inp.value = ''  # Réinitialisation de l'entrée utilisateur après soumission
        # Invocation de l'AgentExecutor pour traiter la requête et générer une réponse
        result = self.qa.invoke({"input": query})
        self.answer = result['output']  
        # Ajout des panneaux d'affichage à la liste panels pour l'utilisateur et la réponse du chatbot
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),  # Affichage de la requête utilisateur
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))  # Affichage de la réponse
        ])
        # Retourne une boîte avec défilement pour afficher la conversation
        return pn.WidgetBox(*self.panels, scroll=True)

    # Fonction pour effacer l'historique de la conversation
    def clr_history(self, count=0):
        self.chat_history = []  # Réinitialisation de l'historique
        return 

In [ ]:
# Création d'une instance de la classe cbfs avec les outils définis
cb = cbfs(tools)

# Widget d'entrée pour le texte utilisateur
inp = pn.widgets.TextInput(placeholder='Enter text here…')

# Liaison de la fonction convchain avec le widget d'entrée
conversation = pn.bind(cb.convchain, inp)

# Définition de l'onglet 'Conversation' de l'interface
tab1 = pn.Column(
    pn.Row(inp),  # Champ de texte d'entrée
    pn.layout.Divider(),  # Séparateur visuel (ligne)
    pn.panel(conversation, loading_indicator=True, height=400),  # Panneau de conversation avec indicateur de chargement
    pn.layout.Divider(),  # Séparateur visuel (ligne)
)

# Création du tableau de bord final avec le titre et l'onglet
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),  # Titre de la page
    pn.Tabs(('Conversation', tab1))  # Onglet pour la conversation
)

# Affiche le tableau de bord
dashboard